In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [7]:
import os 
from pinecone import Pinecone, ServerlessSpec
import time

pc = Pinecone(api_key=os.environ.get("P_API_KEY"))

In [8]:
index_name = "rag-electronics"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )
    # Wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# See that it is empty
print("Index before upsert:")
print(pc.Index(index_name).describe_index_stats())
print("\n")

Index before upsert:
{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}




In [9]:
index = pc.Index(index_name)
index

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ.get("HF_TOKEN") == os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
import nltk 
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aks19\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aks19\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aks19\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\aks19\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [12]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()
bm25_encoder

In [13]:
sentences = [
    "In 2023 i visited paris",
    "In 2024 i visited swiss",
    "In 2024 i visited dubai",
]

# Fit the encoder
bm25_encoder.fit(sentences)

# Save the encoder
bm25_encoder.dump("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 144.61it/s]


In [14]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index = index)

In [15]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001E48B6491E0>, index=<pinecone.data.index.Index object at 0x000001E48B69DAB0>)

In [16]:
retriever.add_texts([
    "In 2023 i visited paris",
    "In 2020 i visited swiss",
    "In 2024 i visited dubai",
    "In 2025 i will visit australia"
])

100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


In [17]:
retriever.invoke("what city will i visit in 2025")

[Document(metadata={'score': 0.479632676}, page_content='In 2025 i will visit australia'),
 Document(metadata={'score': 0.310058475}, page_content='In 2023 i visited paris'),
 Document(metadata={'score': 0.292563498}, page_content='In 2024 i visited dubai'),
 Document(metadata={'score': 0.272536516}, page_content='In 2024 i visited swiss')]

In [ ]:
from langchain.llms import HuggingFacePipeline

In [19]:
import os
hf_token = os.getenv("HF_TOKEN")

In [ ]:
from langchain.llms import HuggingFaceEndpoint
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv

# Load Hugging Face API key
load_dotenv()
hf_token = os.getenv("HF_TOKEN")


llm = HuggingFaceEndpoint(
    endpoint_url="https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=hf_token,
    temperature=0.1,
    top_p=0.9
)

prompt_template = """
You are a helpful AI assistant that answers questions based on the provided context.

Context:
{context}

Question: {question}

Answer the question based only on the provided context. If the information isn't in the context, say "I don't have information about that in my records."

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create RetrievalQA pipeline
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever,chain_type_kwargs={"prompt": PROMPT})

# Test query
response = qa_chain.run("Where did I travel in 2025?")
print(response)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
c:\Users\aks19\anaconda3\envs\rag_e\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Based on the context provided, you are scheduled to visit Australia in 2025.


In [28]:
response = qa_chain.run("Where did I travel in 2025?")
print(response)

c:\Users\aks19\anaconda3\envs\rag_e\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Based on the context provided, you are scheduled to visit Australia in 2025.
